In [1]:
pip install dash pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\balab\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

# Load the dataset
data = pd.read_csv('dataset.csv')
print(data.head())


        State_UT Fiscal Year                       Scheme  \
0  Uttar Pradesh     2015-16  Lohiya Grameen Aawas Yojana   
1  Uttar Pradesh     2015-16  Lohiya Grameen Aawas Yojana   
2  Uttar Pradesh     2015-16  Lohiya Grameen Aawas Yojana   
3  Uttar Pradesh     2016-17  Lohiya Grameen Aawas Yojana   
4  Uttar Pradesh     2016-17  Lohiya Grameen Aawas Yojana   

                                  Scheme Description           Estimate  \
0  Lohiya Awas yojana is social welfare programme...   Budget Estimates   
1  Lohiya Awas yojana is social welfare programme...  Revised Estimates   
2  Lohiya Awas yojana is social welfare programme...            Actuals   
3  Lohiya Awas yojana is social welfare programme...   Budget Estimates   
4  Lohiya Awas yojana is social welfare programme...  Revised Estimates   

    Value          Unit  
0   740.0  In Rs. Crore  
1   740.0  In Rs. Crore  
2   733.0  In Rs. Crore  
3  1361.0  In Rs. Crore  
4  1420.0  In Rs. Crore  


In [ ]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Load dataset
data = pd.read_csv('dataset.csv')

# Data cleaning
data['Value'] = pd.to_numeric(data['Value'], errors='coerce')
data = data.dropna(subset=['Value'])
data['Estimate'] = data['Estimate'].str.strip().str.lower()

# Create Dash app
app = Dash(__name__)

# Prepare dropdown options
scheme_options = [{'label': scheme, 'value': scheme} for scheme in data['Scheme'].unique()]

# Layout with user interface improvements
app.layout = html.Div(
    style={'font-family': 'Arial, sans-serif', 'padding': '30px', 'background-color': '#f7f7f7'},  # Background and padding
    children=[
        html.H1("Social Empowerment Schemes Dashboard", style={'text-align': 'center', 'color': '#333'}),

        # Dropdown for Scheme Selection
        html.Div([
            html.Label("Select Scheme:", style={'font-size': '18px', 'font-weight': 'bold', 'color': '#444'}),
            dcc.Dropdown(
                id='scheme-dropdown',
                options=scheme_options,
                value=data['Scheme'].unique()[0],
                clearable=False,
                style={'width': '60%', 'margin': '0 auto', 'display': 'block'}
            ),
        ], style={'text-align': 'center', 'margin-bottom': '40px'}),

        # Visualization: Reach
        html.Div([
            dcc.Graph(id='reach-graph', style={'height': '400px'}),
        ], style={'margin-bottom': '40px'}),

        # Visualization: Progress
        html.Div([
            dcc.Graph(id='progress-graph', style={'height': '400px'}),
        ], style={'margin-bottom': '40px'}),

        # Visualization: Updates
        html.Div([
            dcc.Graph(id='updates-graph', style={'height': '400px'}),
        ], style={'margin-bottom': '40px'})
    ]
)

@app.callback(
    [Output('reach-graph', 'figure'),
     Output('progress-graph', 'figure'),
     Output('updates-graph', 'figure')],
    [Input('scheme-dropdown', 'value')]
)
def update_graphs(selected_scheme):
    # Filter data for the selected scheme
    filtered_data = data[data['Scheme'] == selected_scheme]

    # Reach Visualization (Updated to Grouped Bar Chart)
    filtered_data['State_Fiscal'] = filtered_data['State_UT'] + " (" + filtered_data['Fiscal Year'] + ")"
    reach_fig = px.bar(
        filtered_data,
        x='Fiscal Year', y='Value',
        color='State_UT',  # Distinguish by state
        title=f"Reach of {selected_scheme} (by State and Year)",
        labels={'Value': 'Beneficiaries (in ₹ Crores)', 'State_UT': 'State/UT', 'Fiscal Year': 'Year'},
        barmode='group',  # Group bars by state within each fiscal year
        color_discrete_sequence=px.colors.sequential.Plasma  # Custom color palette
    )
    
    # Customize layout and style for the bar chart
    reach_fig.update_layout(
        xaxis=dict(tickangle=45),  # Rotate the x-axis labels for better readability
        yaxis=dict(title='Amount (in ₹ Crores)', titlefont=dict(size=14)),
        title=f"Reach of {selected_scheme} (State and Year-wise)",
        plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
        paper_bgcolor='#f4f4f4',  # Light gray background for the chart
        font=dict(size=12, color='black')  # Custom font size and color
    )

    # Progress Visualization (Updated with Markers)
    progress_fig = px.line(
        filtered_data,
        x='Fiscal Year', y='Value',
        color='Estimate',
        markers=True,  # Adding markers for clarity
        title=f"Progress of {selected_scheme} (Yearly)",
        labels={'Value': 'Amount (in ₹ Crores)', 'Fiscal Year': 'Year'},
        line_shape='spline'  # Smooth line curve for better appearance
    )

    # Updates Visualization (Updated with Dynamic Size and Color)
    updates_fig = px.scatter(
        filtered_data,
        x='Fiscal Year', y='Value',
        color='Estimate',
        size='Value',  # Adjust size based on 'Value' for scale of updates
        title=f"Updates for {selected_scheme} (Milestones)",
        labels={'Value': 'Amount (in ₹ Crores)', 'Fiscal Year': 'Year'},
        color_continuous_scale='Viridis'  # Color scale for continuous color mapping
    )

    return reach_fig, progress_fig, updates_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\balab\AppData\Local\Temp\ipykernel_31772\1570493166.py:65: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\balab\AppData\Local\Temp\ipykernel_31772\1570493166.py:65: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

